In [51]:
from __future__ import print_function, division
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, sampler

from torchvision import transforms, utils

import numpy as np
from skimage import io, transform
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Set up hyperparameters and train settings
IN_KERNEL = os.environ.get('KAGGLE_WORKING_DIR') is not None
MIN_SAMPLES_PER_CLASS = 50
BATCH_SIZE = 512
LEARNING_RATE = 1e-3
LR_STEP = 3
LR_FACTOR = 0.5
NUM_WORKERS = multiprocessing.cpu_count()
MAX_STEPS_PER_EPOCH = 15000
NUM_EPOCHS = 2 ** 32
LOG_FREQ = 500
NUM_TOP_PREDICTS = 20
TIME_LIMIT = 9 * 60 * 60

dtype = torch.cuda.FloatTensor
device = torch.device("cuda:0")

In [53]:
# Loading Train Dataset

class LandmarksDataset(Dataset):
    """Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0]) #INCORRECT, NEEDS TO FIT TRAIN FOLDER NAMING
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 2]
        sample = {'image': image, 'class': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample
    

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'landmarks': torch.from_numpy(landmarks)}
    

landmarks_dataset = LandmarksDataset(csv_file = 'train.csv',
                                   root_dir = 'train/',
                                   transform = ToTensor()
                                  )

#Confirm shapes and labels are of correct form
for i in range(len(landmarks_dataset)):
    sample = landmarks_dataset[i]

    print(i, sample['image'].size(), sample['landmarks'])

    if i == 3:
        break
        
class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start=0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples
        
loader_train = DataLoader(landmarks_dataset, batch_size=4,
                        shuffle=True, num_workers=4, sampler)





In [52]:
# Define Model Architecture
model = nn.Sequential(
    nn.Conv2d(in_channels=, out_channels=224, kernel_size=5, stride=1, padding=0),
    nn.ReLU(),
    
)


Sequential(
  (0): Conv2d(224, 224, kernel_size=(5, 5), stride=(1, 1))
  (1): Linear(in_features=1, out_features=1, bias=True)
  (2): ReLU()
)

In [56]:
# Prediction
a = torch.Tensor(range(224))
model(a)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 224, but got 1-dimensional input of size [224] instead